# Transformation of plasmids in 96-well format (OT thermocycler version)

## Protocol Initialization
Ensure you have calibrated the pipettes before doing this protocol


## Before you proceed
This protocol requires the use of the thermocycler lid, which is not easily accessible through jupyter notebooks.

To be able to access the lid, go to the notebook file system tab, then go to New > Terminal and run the command:

```
systemctl stop opentrons-robot-server
```

To re-connect the robot to the app, run:

```
systemctl start opentrons-robot-server
```

This will disconnect the robot from the opentrons app so that only this protocol is able to access it. This is also explained in our [Jupyter Notebook protocols tutorial](https://openplant.github.io/openplant_automation_protocols/Tutorials/Jupyter%20Notebooks/)

In [ ]:
from opentrons import simulate, execute
import json
import time

- When running the protocol with a robot, replace "simulate" below with "execute"

In [ ]:
protocol = simulate.get_protocol_api('2.13')
protocol.home()

## Declaring Labware
- **Left mount**: An 8-channel p20 pipette
- **Right mount**: An 8-channel p300 pipette
- **Slots 7, 8, 10, 11 (don't place the plate here yet)**: Competent cell plate on thermocycler module (96-well PCR plate)
- **Slot 1**: DNA plate (96-well PCR plate)
- **Slot 3**: 12-well reservoir for LB
- **Slots 6 and 9**: Tipracks for both pipettes

Replace with appropriate labware codes in the cell below if needed
If you need to use custom labware, use our [Jupyter Notebook protocols tutorial](https://openplant.github.io/openplant_automation_protocols/Tutorials/Jupyter%20Notebooks/)

In [ ]:
dna_plate = protocol.load_labware('biorad_96_wellplate_200ul_pcr', '1')

thermocycler = protocol.load_module('thermocycler module')
cells_plate = thermocycler.load_labware('biorad_96_wellplate_200ul_pcr')

# Here using 12-well reservoir for a max volume of 20mL.
# Since a 96-well run would use almost 18mL, it might be advisable to use a 1-well reservoir 
with open('custom_labware/enzymax_12_reservoir_20ml.json') as labware_file:
    labware_def = json.load(labware_file)
    lb_res = protocol.load_labware_from_definition(labware_def, 3)

tipracks_20 = [protocol.load_labware('opentrons_96_tiprack_20ul', '9')]
tipracks_300 = [protocol.load_labware('opentrons_96_tiprack_300ul', '6')]  

p20_8 = protocol.load_instrument('p20_multi_gen2', 'left', tip_racks=tipracks_20)
p300_8 = protocol.load_instrument('p300_multi_gen2', 'right', tip_racks=tipracks_300)

# Define the number of samples here
- Run the next cell and fill out the prompts
- You can use from 1 to 96 samples. Do note that using non-multiples of 8 will still make the robot use a whole column of pipette tips.

In [ ]:
samples_no = int(input("Number of samples: "))
final_volume = int(input("What volume do you want the cells to be in at the end? (in ul, max 200) "))

if samples_no - 1 in range(96):
    columns_no = samples_no//8
    if samples_no % 8 > 0:
        columns_no += 1
else:
    samples_no = 0
    print('Invalid number of samples')

## Labware Calibration

Change the x, y, and z variables (**in mm**) in the "set_offset" functions and run each cell repeatedly until the positioning is correct for that piece of labware. You will have to use a "dummy" plate to calibrate the labware that is not on the deck yet (competent cells).

### Tipracks:

In [ ]:
protocol.comment('--- CALIBRATION ---')

In [ ]:
tipracks_20[0].set_offset(x=0.0,y=0.0,z=0.0)
p20_8.move_to(tipracks_20[0]['A1'].top())

In [ ]:
tipracks_300[0].set_offset(x=0.0,y=0.0,z=0.0)
p300_8.move_to(tipracks_300[0]['A1'].top())

### Plates/Racks:

In [ ]:
p20_8.pick_up_tip()

In [ ]:
dna_plate.set_offset(x=0.0,y=0.0,z=0.0)
p20_8.move_to(dna_plate['A1'].top())

- Use a dummy plate to calibrate the cell below

In [ ]:
cells_plate.set_offset(x=0.0,y=0.0,z=0.0)
p20_8.move_to(cells_plate['A1'].top())

In [ ]:
lb_res.set_offset(x=0.0,y=0.0,z=0.0)
p20_8.move_to(lb_res['A1'].top())

In [ ]:
p20_8.return_tip()
p20_8.reset_tipracks()

## Protocol Execution
Flash freeze and heat shock

In [ ]:
print("Setting temperature to 4°C")
thermocycler.set_block_temperature(temperature=4)
print('Done')

- Place the plate with competent cells on the thermocycler block
- Run the next cell immediately

In [ ]:
protocol.comment('--- TRANSFERRING DNA INTO COMPETENT CELLS ---')
print(protocol.commands()[-1])
print('10 minutes wait started at', time.strftime("%H:%M:%S", time.localtime()))
protocol.delay(minutes=10)
for col in range(columns_no):
    print('Transferring column', col + 1)
    p20_8.pick_up_tip()
    p20_8.aspirate(5, dna_plate.rows()[0][col])
    p20_8.dispense(5, cells_plate.rows()[0][col].bottom(z=1.5))
    p20_8.mix(3, 10, cells_plate.rows()[0][col])
    p20_8.touch_tip()
    p20_8.drop_tip()

thermocycler.close_lid()
print('20 minutes wait started at', time.strftime("%H:%M:%S", time.localtime()))
protocol.delay(minutes=20)
thermocycler.set_block_temperature(temperature=42)
protocol.delay(seconds=30)
thermocycler.set_block_temperature(temperature=4)
thermocycler.open_lid()
    
print("Done")

In [ ]:
protocol.comment('--- TRANSFERRING LB INTO CELLS PLATE ---')
print(protocol.commands()[-1])

p300_8.pick_up_tip()
for col in range(columns_no):
    print('Transferring column', col + 1)
    p300_8.aspirate(final_volume - 15, lb_res.rows()[0][0])
    p300_8.dispense(final_volume - 15, cells_plate.rows()[0][col].top()) # I removed the touch tip here, and am dispensing from the top of the well to avoid contamination
    p300_8.move_to(lb_res.rows()[0][0].top())
    p300_8.blow_out()
p300_8.drop_tip()
print('Done')

- Place transformation plate in an incbator at 37°C for one hour
- Prepare the deck and robot for the plating protocol

In [ ]:
thermocycler.deactivate()
protocol.home()

In [ ]:
protocol.commands()